In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Data

In [2]:
# Load and preprocess data

# replace below with your path
path = 'xnas-itch-20240822.mbp-1.csv'
mbp_csv = pd.read_csv(path, parse_dates=['ts_recv', 'ts_event'])

# Calculate the log-arithmetic middle price
#mbp_csv['log_arith_middle_price'] = (np.log(mbp_csv['bid_px_00']) + np.log(mbp_csv['ask_px_00'])) / 2
mbp_csv['log_arith_middle_price'] = np.log((mbp_csv['bid_px_00']+mbp_csv['ask_px_00'])/2)

# Subtract estimation fo noise from log_returns to eliminate negative TSRVs
smoothed_log_returns = mbp_csv['log_arith_middle_price'].rolling(window=10).mean()

mbp_csv['smoothed_log_returns'] = mbp_csv['log_arith_middle_price'] - smoothed_log_returns

# Step 4: Calculate the variance of the noise
mbp_csv['smoothed_variance'] = (noise_estimation ** 2).sum()

# Step 5: Estimate the true variance by subtracting noise variance from total variance
true_variance = high_freq_variance - noise_variance


NameError: name 'noise_estimation' is not defined

In [3]:
# debugging
negative_log_price_df = mbp_csv[mbp_csv['log_arith_middle_price'] < 0]

print(negative_log_price_df)


Empty DataFrame
Columns: [ts_recv, ts_event, rtype, publisher_id, instrument_id, action, side, depth, price, size, flags, ts_in_delta, sequence, bid_px_00, ask_px_00, bid_sz_00, ask_sz_00, bid_ct_00, ask_ct_00, symbol, log_arith_middle_price, smoothed_log_returns]
Index: []

[0 rows x 22 columns]


In [4]:
mbp_csv

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,...,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,log_arith_middle_price,smoothed_log_returns
0,2024-08-22 13:00:00.148489656+00:00,2024-08-22 13:00:00.148324259+00:00,1,2,11667,C,A,0,129.93,600,...,11891132,129.80,129.93,899,306,4,3,NVDA,4.866495,NaN
1,2024-08-22 13:00:01.015421988+00:00,2024-08-22 13:00:01.015255234+00:00,1,2,11667,T,N,0,129.89,700,...,11895306,129.80,129.93,899,306,4,3,NVDA,4.866495,NaN
2,2024-08-22 13:00:06.026216498+00:00,2024-08-22 13:00:06.026050629+00:00,1,2,11667,A,A,0,129.91,4,...,11908757,129.80,129.91,899,4,4,1,NVDA,4.866418,NaN
3,2024-08-22 13:00:08.021418483+00:00,2024-08-22 13:00:08.021252273+00:00,1,2,11667,A,B,0,129.81,100,...,11913875,129.81,129.91,100,4,1,1,NVDA,4.866457,NaN
4,2024-08-22 13:00:08.378809454+00:00,2024-08-22 13:00:08.378639226+00:00,1,2,11667,T,N,0,129.89,100,...,11914309,129.81,129.91,100,4,1,1,NVDA,4.866457,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387422,2024-08-22 20:59:46.873416500+00:00,2024-08-22 20:59:46.873250383+00:00,1,2,11667,A,B,0,124.33,1,...,631379773,124.33,124.42,1,100,1,1,NVDA,4.823301,0.000245
3387423,2024-08-22 20:59:50.256406269+00:00,2024-08-22 20:59:50.256237949+00:00,1,2,11667,T,A,0,124.33,1,...,631381505,124.33,124.42,1,100,1,1,NVDA,4.823301,0.000209
3387424,2024-08-22 20:59:50.256406269+00:00,2024-08-22 20:59:50.256237949+00:00,1,2,11667,T,N,0,124.33,49,...,631381506,124.30,124.42,826,100,5,1,NVDA,4.823181,0.000064
3387425,2024-08-22 20:59:50.363312798+00:00,2024-08-22 20:59:50.363146531+00:00,1,2,11667,A,B,0,124.32,2,...,631381570,124.32,124.42,2,100,1,1,NVDA,4.823261,0.000113


Helper Functions

In [5]:
# Function to compute subsampled variance [X, X]^K_T
def compute_subsampled_variance(log_returns, K):
    n = len(log_returns)
    subsampled_variance = 0

    log_returns = list(log_returns)

    for i in range(n - K):
        pt1 = log_returns[i + K]
        pt2 = log_returns[i]
        subsampled_variance += (pt1 - pt2)**2

    subsampled_variance /= K
    return subsampled_variance

In [6]:
# Function to compute full sample variance [X, X]^{All}_T
def compute_full_sample_variance(log_returns):
    return np.sum(np.diff(log_returns)**2)

In [7]:
# Function to compute TSRV
def compute_TSRV(log_returns, diffs, K):
    n = len(log_returns)
    subsampled_variance = compute_subsampled_variance(log_returns, K)
    full_sample_variance = compute_full_sample_variance(log_returns)
    z = (n - K + 1) / K
    tsrv = (1 - z/n)**(-1) * (subsampled_variance - (z/n) * full_sample_variance)
    diffs.append(abs(subsampled_variance - full_sample_variance))
    if tsrv < 0:
        print('diff between ', abs(subsampled_variance - full_sample_variance))
        print('the negative value')
        print('sub ', subsampled_variance)
        print('full ', full_sample_variance)
        print('z ', z)
    return tsrv, diffs

In [8]:
# Function to compute realized volatility for different intervals
def compute_realized_volatility(df, symbol):
    results = {}
    #calculate for 10 min
    df_resampled = df[df['symbol'] == symbol].set_index('ts_event').resample('10T')
    tsrvs = []
    diffs = []
    flag = 0
    for _, group in df_resampled:
        log_returns = group['log_arith_middle_price'].values
        # >= 5 produces negative tsrv for NVDA
        tsrv, diffs = compute_TSRV(log_returns, diffs, K=4)
        if tsrv < 0: # negative
            #print(group['log_arith_middle_price'].columns)
            group['log_arith_middle_price'].to_clipboard()
            print('negative tsrv, ', tsrv)
            flag = 1
            continue
        tsrvs.append(np.sqrt(tsrv))
    results['10 min'] = np.mean(tsrvs)

    if flag == 1:
        neg_value_diff = float((max(diffs)))
        diffs.remove(neg_value_diff)
        secnd = max(diffs)
        print('diff between neg and second largest diff')
        print(abs(neg_value_diff - secnd))


    return results


Calculate Realised Volatility

In [9]:
# Compute realized volatility for NVDA and CAKE
nvda_results = compute_realized_volatility(mbp_csv, 'NVDA')
cake_results = compute_realized_volatility(mbp_csv, 'CAKE')

print("Realized Volatility for NVDA:")
for interval, rv in nvda_results.items():
    print(f"{interval}: {rv}")

print("\nRealized Volatility for CAKE:")
for interval, rv in cake_results.items():    print(f"{interval}: {rv}")

/tmp/ipykernel_13855/880205488.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df[df['symbol'] == symbol].set_index('ts_event').resample('10T')


Realized Volatility for NVDA:
10 min: 0.003746863343946456

Realized Volatility for CAKE:
10 min: 0.002435693504315161


In [10]:
# high-frequency noise estimate
def compute_high_frequency_noise(log_returns, high_freq_K):
    return compute_subsampled_variance(log_returns, high_freq_K) / high_freq_K

# TSRV with noise correction
def compute_TSRV_with_noise_correction(log_returns, diffs, K, high_freq_K):
    n = len(log_returns)

    # Lower subsample freq
    subsampled_variance = compute_subsampled_variance(log_returns, K)

    # Full sample variance
    full_sample_variance = compute_full_sample_variance(log_returns)

    # Higher subsample freq for noise estimate
    high_freq_noise = compute_high_frequency_noise(log_returns, high_freq_K)

    # Calculate TSRV without noise adjustment
    z = (n - K + 1) / K
    tsrv = (1 - 2/n) * (subsampled_variance / K) - (z / n) * full_sample_variance

    # From discussion slides - adjust TSRV
    scaling_factor = high_freq_K / n
    tsrv_corrected = tsrv - (scaling_factor * high_freq_noise)

    diffs.append(abs(subsampled_variance - full_sample_variance))

    return tsrv_corrected, diffs

def compute_realized_volatility_with_noise(df, symbol):
    results = {}
    diffs = []

    df_resampled = df[df['symbol'] == symbol].set_index('ts_event').resample('10T')

    for _, group in df_resampled:
        log_returns = group['log_arith_middle_price'].values

        # Calculate TSRV with noise correction
        tsrv, diffs = compute_TSRV_with_noise_correction(log_returns, diffs, K=4, high_freq_K=4)

        # Check for negative TSRV and continue if found
        if tsrv < 0:
            print('Negative TSRV:', tsrv)
            continue

        # Append the TSRV result
        results['10 min'] = np.sqrt(tsrv)

    return results


In [11]:
realized_volatility_results = compute_realized_volatility_with_noise(mbp_csv, 'NVDA')
print(realized_volatility_results)

/tmp/ipykernel_13855/3261111476.py:34: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df[df['symbol'] == symbol].set_index('ts_event').resample('10T')


Negative TSRV: -3.497410358421655e-07
Negative TSRV: -5.322782912021893e-07
Negative TSRV: -1.714356314498699e-06
Negative TSRV: -1.6006663141668193e-06
Negative TSRV: -5.745169918673796e-07
Negative TSRV: -6.148069088349416e-07
Negative TSRV: -8.643293004468528e-07
Negative TSRV: -6.671828527899834e-07
Negative TSRV: -5.481937033507532e-07
Negative TSRV: -2.612523339868478e-07
Negative TSRV: -3.3109545287444655e-07
Negative TSRV: -2.984133619222748e-07
Negative TSRV: -4.354216149454592e-07
Negative TSRV: -3.5005224324243843e-07
Negative TSRV: -3.73306250782593e-07
Negative TSRV: -6.440232261681012e-07
Negative TSRV: -1.5476830794187968e-06
Negative TSRV: -3.7422194718859103e-07
Negative TSRV: -3.47631450694595e-07
Negative TSRV: -3.742950084448688e-07
Negative TSRV: -6.603146710026702e-07
Negative TSRV: -4.081548919465287e-07
Negative TSRV: -5.308343063335118e-07
Negative TSRV: -5.494562908991499e-07
Negative TSRV: -5.268629674151555e-07
Negative TSRV: -4.598600021606208e-07
Negative 